In [1]:
import glob
import os

import pandas as pd
import xarray as xr
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components
import numpy as np
import netCDF4 as nc
import h5py

In [10]:
ex_fpath = r"\\wsl$\Ubuntu-18.04\home\arlenlex\LIGGGHTS_SEAICE\lexi_tests\floe_testing\shear_floe\post3d\dump0000005904.liggghts"
ex_bpath = r"\\wsl$\Ubuntu-18.04\home\arlenlex\LIGGGHTS_SEAICE\lexi_tests\floe_testing\shear_floe\post3d\bfc0000005904.bond"
ex_directory = r"\\wsl$\Ubuntu-18.04\home\arlenlex\LIGGGHTS_SEAICE\lexi_tests\floe_testing\shear_floe\post3d"
atom_output = r"c:\Users\arlenlex\Documents\LIGGGHTS Visualization\python visualization\test_output_data\atom.nc"
bond_output = r"c:\Users\arlenlex\Documents\LIGGGHTS Visualization\python visualization\test_output_data\bond.nc"

In [3]:
def process_atom_dump_file(filepath: os.PathLike):
    """Read atom dump file and convert to xarray Dataset."""
    with open(filepath, 'r') as file:
        header_lines = [next(file).strip() for _ in range(9)]  # Read only the first 9 lines

    # Extract specific information from the lines
    timestep = int(header_lines[1])
    number_of_atoms = int(header_lines[3])
    column_names = header_lines[8].split()[2:]  # Assumes the 9th line has the relevant data

    # read into pandas dataframe
    df = pd.read_csv(filepath, index_col="id", sep = r'\s+', skiprows=9, names=column_names)
    
    # convert to xarray, add timestep dim and attributes 
    ds = df.to_xarray()
    ds = ds.expand_dims(timestep=[timestep])
    ds = ds.assign_attrs(number_of_atoms=number_of_atoms)

    return ds

def get_atom_ds(directory_path: os.PathLike, output_path: os.PathLike, return_ds = False):
    """Process all dump files from a directory and concatenate into xarray Dataset."""
    filepaths = sorted(glob.glob(os.path.join(directory_path, 'dump*.liggghts')))
    datasets = (process_atom_dump_file(fp) for fp in filepaths)
    ds = xr.concat(datasets, dim="timestep") # modify for lost atoms; i think this is in the defaults
    ds.to_netcdf(output_path)
    print(f'Saved {os.path.basename(output_path)}.')
    if return_ds:
        return ds


In [17]:
def process_bond_dump_file(filepath: os.PathLike, column_names):
    with open(filepath, 'r') as file:
        header_lines = [next(file).strip() for _ in range(9)]  # Read only the first 9 lines

    # Extract specific information from the lines
    timestep = int(header_lines[1])
    number_of_bonds = int(header_lines[3])
    column_names = column_names.split()

    # read into pandas dataframe
    df = pd.read_csv(filepath, sep = r'\s+', skiprows=9, names=column_names)

    # convert to xarray, add timestep dim and attributes 
    ds = df.to_xarray()
    ds = ds.assign_attrs(number_of_bonds=number_of_bonds)
    ds = ds.assign_attrs(timestep=timestep)
    return ds

def get_bond_ds(directory_path: os.PathLike, output_path: os.PathLike, column_names, return_ds = False):
    filepaths = sorted(glob.glob(os.path.join(directory_path, 'bfc*.bond')))
    with nc.Dataset(output_path, 'w', format = 'NETCDF4') as f:
        for fpath in filepaths:
            data = process_bond_dump_file(fpath, column_names)
            timestep = data.attrs['timestep']
            nbonds = data.attrs['number_of_bonds']
            group = f.createGroup(f'{timestep}')
            group.createDimension('index', nbonds)
            for v in list(data.variables.keys()):
                variable = group.createVariable(v, 'f4', ('index',))
                variable[:] = data[v]
    print(f'Saved {os.path.basename(output_path)}.')
    if return_ds:
        return nc.Dataset(output_path, 'w')

In [6]:
column_names = 'batom1x batom1y batom1z batom2x batom2y batom2z batom1 batom2 bbondbroken bforceX bforceY bforceZ btorqueX btorqueY btorqueZ beqdist'

In [11]:
get_atom_ds(ex_directory, atom_output)

Saved atom.nc.


In [18]:
get_bond_ds(ex_directory, bond_output, column_names)

Saved bond.nc.


# bonds
plan is for every timestep, save a netcdf file that has variables: number_of_bonds, number_of_connected_components, labels... if i wanted to see the bond force magnitude, that'd make this harder... 

In [91]:
filepath = ex_bpath
with open(filepath, 'r') as file:
    header_lines = [next(file).strip() for _ in range(9)]  # Read only the first 9 lines

# Extract specific information from the lines
timestep = int(header_lines[1])
number_of_bonds = int(header_lines[3])
column_names = header_lines[8].split()[2:]  # Assumes the 9th line has the relevant data

# read into pandas dataframe
df = pd.read_csv(filepath, sep = r'\s+', skiprows=9, names=column_names)

lexi method

In [ ]:
atom1_id_dx = 6
atom2_id_dx = 7
bond_status_idx = 8
keys = df.keys()

# remove all rows where bonds are broken
df_active = df[df.iloc[:, bond_status_idx] != 1] 
del df

# get max atom id
max_atom_id = (df_active.iloc[:, [atom1_id_dx, atom2_id_dx]]).max().max()

# construct dense matrix
dense_matrix = np.zeros((max_atom_id, max_atom_id), dtype ='int')

for __, row in df_active.iterrows():
    i, j = (row.iloc[atom1_id_dx]).astype(int), (row.iloc[atom2_id_dx]).astype(int)
    dense_matrix[i - 1, j - 1] = 1 # subtract 1 to account for python indexing

# make symmetric
dense_matrix = np.logical_or(dense_matrix, dense_matrix.T).astype(int)

# construct sparse matrix
bond_graph = csr_matrix(dense_matrix)
del dense_matrix

# get number of connected components & their labels
n_components, labels = connected_components(csgraph=bond_graph, directed=False, return_labels=True)
del bond_graph

# get the size of the connected components
component_sizes = np.bincount(labels)

# create the xarray dataset

# ds

In [98]:
ds = xr.Dataset(
    data_vars=dict(
        number_of_bonds=("timestep", [number_of_bonds]),
        number_of_components=("timestep", [n_components]),
        component_sizes=("timestep", [n_components]),
        ),
    )

In [99]:
datas

<xarray.Dataset> Size: 16B
Dimensions:               (timestep: 1)
Dimensions without coordinates: timestep
Data variables:
    number_of_bonds       (timestep) int64 8B 8168
    number_of_components  (timestep) int64 8B 9

In [43]:
def process_bond_dump_file(filepath: os.PathLike, atom1_id_idx = 0, atom2_id_idx = 1, bond_status_idx = 2):
    """Read bond dump file and convert to sparse matrices for each param."""
    with open(filepath, 'r') as file:
        header_lines = [next(file).strip() for _ in range(9)]  # Read only the first 9 lines

    # Extract specific information from the lines
    timestep = int(header_lines[1])
    number_of_bonds = int(header_lines[3])
    column_names = header_lines[8].split()[2:]  # Assumes the 9th line has the relevant data

    # read into pandas dataframe
    df = pd.read_csv(filepath, sep = r'\s+', skiprows=9, names=column_names)
    
    # convert to xarray, add timestep dim and attributes 
    ds = df.to_xarray()
    ds = ds.expand_dims(timestep=[timestep])
    ds['number_of_bonds'] = number_of_bonds

    return ds

# this is probs not what i want to do...
def get_bond_ds(directory_path: os.PathLike):
    """Process all bond dump files from a directory and concatenate into xarray Dataset."""
    filepaths = sorted(glob.glob(os.path.join(directory_path, 'bfc*.bond')))
    datasets = (process_bond_dump_file(fp) for fp in filepaths)
    ds = xr.concat(datasets, dim="timestep") # modify for lost atoms; i think this is in the defaults
    return ds

In [44]:
ds = get_bond_ds(ex_directory)

In [45]:
ds

<xarray.Dataset> Size: 127MB
Dimensions:          (timestep: 121, index: 8168)
Coordinates:
  * index            (index) int64 65kB 0 1 2 3 4 5 ... 8163 8164 8165 8166 8167
  * timestep         (timestep) int64 968B 246 492 738 984 ... 29274 29520 29766
Data variables: (12/17)
    c_bc[1]          (timestep, index) float64 8MB 0.5107 0.5107 ... nan nan
    c_bc[2]          (timestep, index) float64 8MB 0.8834 0.8834 ... nan nan
    c_bc[3]          (timestep, index) float64 8MB 0.0 0.0 0.0 ... nan nan nan
    c_bc[4]          (timestep, index) float64 8MB 1.02 1.531 1.021 ... nan nan
    c_bc[5]          (timestep, index) float64 8MB 3.043e-05 0.8834 ... nan nan
    c_bc[6]          (timestep, index) float64 8MB 0.0 0.0 0.0 ... nan nan nan
    ...               ...
    c_bc[12]         (timestep, index) float64 8MB 0.0 0.0 0.0 ... nan nan nan
    c_bc[13]         (timestep, index) float64 8MB 0.0 0.0 0.0 ... nan nan nan
    c_bc[14]         (timestep, index) float64 8MB 0.0 0.0 0.0 ... nan nan nan
    c_bc[15]         (timestep, index) float64 8MB -683.5 49.68 ... nan nan
    c_bc[16]         (timestep, index) float64 8MB 1.02 1.02 1.02 ... nan nan
    number_of_bonds  (timestep) int64 968B 8168 8168 8168 ... 4418 4368 4368

In [16]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import connected_components, dijkstra

In [39]:
# Example: Create a dense matrix representing a graph
graph = [

[0, 1, 1, 0, 0],

[0, 0, 1, 0, 0],

[0, 0, 0, 0, 0],

[0, 0, 0, 0, 1],

[0, 0, 0, 0, 0]

]

# Convert the dense matrix to a CSR matrix
sparse_matrix = csr_matrix(graph)

# Now, you can use `sparse_matrix` with scipy's csgraph functions
# Example: Find the number of connected components
n_components, labels = connected_components(sparse_matrix)

In [40]:
n_components

2

In [41]:
labels

array([0, 0, 0, 1, 1], dtype=int32)